In [50]:
#!pip install xgboost

In [51]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# # Exclude boolean columns (location_A, location_B, location_C)
# columns_to_scale = x_train.columns.difference(['location_A', 'location_B', 'location_C'])

# scaler = MinMaxScaler()
# x_train[columns_to_scale] = scaler.fit_transform(x_train[columns_to_scale])
# x_test[columns_to_scale] = scaler.transform(x_test[columns_to_scale])


pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [52]:
x_train = pd.read_csv('cleaned_data/x_train_combined.csv')
y_train = pd.read_csv('cleaned_data/y_train_combined.csv')
x_test = pd.read_csv('cleaned_data/x_test_combined.csv')

# Determine the split index
split_index = int(0.875 * len(x_train))

# Split the data into training and validation sets
x_train_df = x_train[:split_index]
y_train_df = y_train[:split_index]
x_valid_df = x_train[split_index:]
y_valid_df = y_train[split_index:]



In [53]:
from sklearn.preprocessing import MinMaxScaler

# Exclude boolean columns (location_A, location_B, location_C)
# columns_to_scale = x_train.columns.difference(['location_A', 'location_B', 'location_C'])

# scaler = MinMaxScaler()
# x_train[columns_to_scale] = scaler.fit_transform(x_train[columns_to_scale])
# x_test[columns_to_scale] = scaler.transform(x_test[columns_to_scale])


In [54]:


dtrain = xgb.DMatrix(x_train_df,label = y_train_df['pv_measurement'])
dval = xgb.DMatrix(x_valid_df,label = y_valid_df['pv_measurement'])
test = xgb.DMatrix(x_test)


In [55]:
xgb_params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    'booster': 'gbtree',
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'alpha': 0.1,
    'lambda': 1,

}
#  Train the model
num_boost_round = 1000  # Maximum number of boosting rounds
early_stopping_rounds = 50  # Stop if validation score doesn't improve for 50 rounds

evals = [(dtrain, 'train'), (dval, 'eval')]
progress = {}


In [56]:
bst = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=evals,
    early_stopping_rounds=early_stopping_rounds,
    evals_result=progress
)

print("Best MAE: {:.2f} with {} rounds".format(
         bst.best_score,
         bst.best_iteration+1))


[0]	train-mae:410.81769	eval-mae:355.02348
[1]	train-mae:391.97774	eval-mae:339.55657
[2]	train-mae:374.25054	eval-mae:324.94830
[3]	train-mae:357.63684	eval-mae:311.05315
[4]	train-mae:341.91711	eval-mae:298.07872
[5]	train-mae:327.01209	eval-mae:285.79752
[6]	train-mae:313.06909	eval-mae:274.27079
[7]	train-mae:299.99073	eval-mae:263.35323
[8]	train-mae:287.64472	eval-mae:253.14326
[9]	train-mae:276.06925	eval-mae:243.46777
[10]	train-mae:265.26259	eval-mae:234.37246
[11]	train-mae:255.10855	eval-mae:225.55454
[12]	train-mae:245.64420	eval-mae:217.29854
[13]	train-mae:236.73054	eval-mae:209.58611
[14]	train-mae:228.37144	eval-mae:202.30888
[15]	train-mae:220.49649	eval-mae:195.30857
[16]	train-mae:213.13562	eval-mae:188.77218
[17]	train-mae:206.20922	eval-mae:182.57907
[18]	train-mae:199.74358	eval-mae:176.63054
[19]	train-mae:193.56542	eval-mae:171.00853
[20]	train-mae:187.85561	eval-mae:165.76284
[21]	train-mae:182.43327	eval-mae:160.91161
[22]	train-mae:177.30652	eval-mae:156.1171

In [57]:
predictions = bst.predict(test)
predictions

array([ 2.2806997 ,  2.3543084 ,  2.9639523 , ...,  1.544247  ,
        0.22856957, -1.0557479 ], dtype=float32)

In [58]:
sample_submission = pd.read_csv('sample_submission.csv')

# Convert the numpy array to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['prediction'])

# Join the 'id' column from sample_submission with the predictions
sample_submission['prediction'] = predictions_df['prediction']

# Save to CSV
sample_submission.to_csv('my_first_submission.csv', index=False)